In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [2]:
df_workout=pd.read_csv("DL_Workout.csv")

In [3]:
feature_cols = [
    "Age",
    "Gender",
    "BMI",
    "Experience_Level",
    "Session_Duration (hours)",
    "Difficulty Level",
    "Reps",
    "strength",
    "cardio",
    "hiit",
    "yoga"
]

X = df_workout[feature_cols].values
y = df_workout["weight_gain_score"].values  # or weight_gain_score


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [5]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


In [6]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_test_t  = torch.tensor(X_test, dtype=torch.float32)
y_test_t  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


In [7]:
class WorkoutRecommender_gain(nn.Module):
    def __init__(self, input_dim):
        super(WorkoutRecommender_gain, self).__init__()

        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.out = nn.Linear(16, 1)
    #fc1 ,fc2=hidden layers
        
        self.relu = nn.ReLU() #for learning non linear patterns
        self.sigmoid = nn.Sigmoid() #for output between 0 and 1

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.out(x))
        return x

In [8]:
model = WorkoutRecommender_gain(input_dim=X_train.shape[1])

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 60
for epoch in range(epochs):
    optimizer.zero_grad()
    preds = model(X_train_t)
    loss = criterion(preds, y_train_t)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")



Epoch 5, Loss: 0.0129
Epoch 10, Loss: 0.0125
Epoch 15, Loss: 0.0122
Epoch 20, Loss: 0.0118
Epoch 25, Loss: 0.0114
Epoch 30, Loss: 0.0110
Epoch 35, Loss: 0.0105
Epoch 40, Loss: 0.0100
Epoch 45, Loss: 0.0094
Epoch 50, Loss: 0.0087
Epoch 55, Loss: 0.0080
Epoch 60, Loss: 0.0073


In [9]:
model.eval()
with torch.no_grad():
    test_preds = model(X_test_t)
    test_loss = criterion(test_preds, y_test_t)

print("Weight Gain Model Test Loss:", test_loss.item())


Weight Gain Model Test Loss: 0.007487759459763765


In [10]:
checkpoint = {
    "model_state_dict": model.state_dict(),
    "input_dim": X.shape[1],
    "feature_cols": feature_cols,          # VERY IMPORTANT
    "scaler": scaler,                       # normalization
}
torch.save(checkpoint, "workout_recommender(gain).pth")


In [16]:

user_input = {
    "age": 30,
    "weight": 70,
    "height": 1.65,
    "gender": 1  , # male
    "experience_level":0
}



In [22]:
def recommend_workouts(user_input, exercise_df, model, scaler, top_n=5):

    bmi = user_input["weight"] / (user_input["height"] ** 2)
    model.eval()
    recommendations = []

    for _, row in exercise_df.iterrows():

        X = [
            user_input["age"],
            user_input["gender"],
            bmi,
            user_input["experience_level"],
            row["Session_Duration (hours)"],
            row["Difficulty Level"],
            row["Reps"],
            row["strength"],
            row["cardio"],
            row["hiit"],
            row["yoga"]
        ]

        X = scaler.transform([X])
        X_t = torch.tensor(X, dtype=torch.float32)

        with torch.no_grad():
            score = model(X_t).item()
        col=["Name of Exercise","target_muscles","focus_area"]
        recommendations.append((row[col], score))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]


In [23]:

top_exercises = recommend_workouts(
    user_input,
    df_workout,
    model,
    scaler,
    top_n=5
)


In [25]:
top_exercises

[(Name of Exercise    Scissors Kicks
  target_muscles       ['lower_abs']
  focus_area                    core
  Name: 8717, dtype: object,
  0.5686420202255249),
 (Name of Exercise                   Flutter Kicks
  target_muscles      ['lower_abs', 'hip_flexors']
  focus_area                                  core
  Name: 1824, dtype: object,
  0.5684894323348999),
 (Name of Exercise                   Kettlebell Swings
  target_muscles      ['glutes', 'hamstrings', 'core']
  focus_area                                  fullbody
  Name: 1577, dtype: object,
  0.568443775177002),
 (Name of Exercise    Scissors Kicks
  target_muscles       ['lower_abs']
  focus_area                    core
  Name: 17696, dtype: object,
  0.5683896541595459),
 (Name of Exercise                        Thrusters
  target_muscles      ['legs', 'shoulders', 'core']
  focus_area                               fullbody
  Name: 8934, dtype: object,
  0.5683654546737671)]